# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
! pip install yfinance numpy pandas scipy XlsxWriter ipython

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import math
from scipy.stats import percentileofscore as score
from statistics import mean
import xlsxwriter
from IPython.display import clear_output

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. But, here we have just taken a list of ~500 stocks in a CSV file taken at the time of building the project.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('sp500.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [4]:
my_columns = ['Ticker',
              'Stock Price',
              'P/E Ratio',
              'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Here to fetch the stock data, we have used `yfinance` API

We can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [5]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    price = data['currentPrice']
    try:
        peRatio = data['currentPrice'] / data['trailingEps']
    except:
        peRatio = np.NaN
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [price],
        'P/E Ratio': [peRatio],
        'Number of Shares to Buy': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(final_dataframe)

    Ticker  Stock Price  P/E Ratio Number of Shares to Buy
0        A     110.1700  28.840314                     N/A
1      AAL      13.2350   3.446615                     N/A
2      AAP      58.9300  10.177893                     N/A
3     AAPL     175.3354  29.418691                     N/A
4     ABBV     153.6200  31.609053                     N/A
..     ...          ...        ...                     ...
498    YUM     125.6800  25.389899                     N/A
499    ZBH     118.1200  49.422594                     N/A
500   ZBRA     231.2250  19.847639                     N/A
501   ZION      34.4700   6.079365                     N/A
502    ZTS     177.7750  37.426316                     N/A

[503 rows x 4 columns]


In [6]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,A,110.1700,28.840314,N/A
1,AAL,13.2350,3.446615,N/A
2,AAP,58.9300,10.177893,N/A
3,AAPL,175.3354,29.418691,N/A
4,ABBV,153.6200,31.609053,N/A
...,...,...,...,...
498,YUM,125.6800,25.389899,N/A
499,ZBH,118.1200,49.422594,N/A
500,ZBRA,231.2250,19.847639,N/A
501,ZION,34.4700,6.079365,N/A


In [7]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
80,CAT,274.58,NaN,N/A


Filling the `NULL` cells of the `P/E Ratio` column with mean value

In [8]:
final_dataframe['P/E Ratio'].fillna(final_dataframe['P/E Ratio'].mean(), inplace=True)

In [9]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy


In [10]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,A,110.1700,28.840314,N/A
1,AAL,13.2350,3.446615,N/A
2,AAP,58.9300,10.177893,N/A
3,AAPL,175.3354,29.418691,N/A
4,ABBV,153.6200,31.609053,N/A
...,...,...,...,...
498,YUM,125.6800,25.389899,N/A
499,ZBH,118.1200,49.422594,N/A
500,ZBRA,231.2250,19.847639,N/A
501,ZION,34.4700,6.079365,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll consider only those stocks whose `P/E Ratio` is positive.

In [11]:
final_dataframe.sort_values('P/E Ratio', ascending=True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['P/E Ratio'] > 0]

In [12]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
1,AAL,13.2350,3.446615,N/A
100,CMA,41.6095,4.343372,N/A
166,EQT,39.3050,4.476651,N/A
206,GM,33.0650,4.618017,N/A
468,VLO,147.8350,5.090737,N/A
...,...,...,...,...
420,STE,220.8550,184.045833,N/A
473,VTR,43.3000,196.818182,N/A
482,WELL,84.2500,324.038462,N/A
431,TAP,64.3300,338.578947,N/A


In [13]:
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [14]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,AAL,13.2350,3.446615,N/A
1,CMA,41.6095,4.343372,N/A
2,EQT,39.3050,4.476651,N/A
3,GM,33.0650,4.618017,N/A
4,VLO,147.8350,5.090737,N/A
5,ALB,170.2701,5.117827,N/A
6,PSX,120.6900,5.204398,N/A
7,UAL,44.2400,5.564780,N/A
8,MPC,155.5600,5.704437,N/A
9,STLD,100.2400,5.804285,N/A


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [15]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [16]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

200000.0

In [17]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,AAL,13.2350,3.446615,15111
1,CMA,41.6095,4.343372,4806
2,EQT,39.3050,4.476651,5088
3,GM,33.0650,4.618017,6048
4,VLO,147.8350,5.090737,1352
5,ALB,170.2701,5.117827,1174
6,PSX,120.6900,5.204398,1657
7,UAL,44.2400,5.564780,4520
8,MPC,155.5600,5.704437,1285
9,STLD,100.2400,5.804285,1995


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

In [18]:
rv_columns = ['Ticker',
              'Stock Price',
              'Number of Shares to Buy',
              'Price-to-Earning Ratio',
              'PE Percentile',
              'Price-to-Book Value',
              'PB Percentile',
              'Price-to-Sales Ratio',
              'PS Percentile',
              'EV/EBITDA',
              'EV/EBITDA Percentile',
              'EV/GP',
              'EV/GP Percentile',
              'RV Score']

rv_dataframe = pd.DataFrame(columns=rv_columns)
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [19]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    financials = yf.Ticker(stock).financials

    try:
        price = data['currentPrice']
    except:
        price = np.NaN

    try:
        eps = data['trailingEps']
    except:
        eps = 1

    try:
        peRatio = price / eps
    except:
        peRatio = np.NaN

    try:
        pbRatio = data['priceToBook']
    except:
        pbRatio = np.NaN

    try:
        marketCap = data['marketCap']
    except:
        marketCap = np.NaN

    try:
        outstandingShares = data['sharesOutstanding']
    except:
        outstandingShares = 1

    try:
        psRatio = marketCap / (outstandingShares * eps)
    except:
        psRatio = np.NaN

    try:
        debt = data['totalDebt']
    except:
        debt = np.NaN

    try:
        cash = data['totalCash']
    except:
        cash = np.NaN

    try:
        ebitda = data['ebitda']
    except:
        ebitda = 1

    try:
        evToEBIDTA = (marketCap + debt - cash) / ebitda
    except:
        evToEBIDTA = np.NaN

    try:
        grossProfit = financials.loc['Gross Profit'][0]
    except:
        grossProfit = 1

    try:
        evToGP = (marketCap + debt - cash) / grossProfit
    except:
        evToGP = np.NaN

    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [price],
        'Number of Shares to Buy': ['N/A'],
        'Price-to-Earning Ratio': [peRatio],
        'PE Percentile': ['N/A'],
        'Price-to-Book Value': [pbRatio],
        'PB Percentile': ['N/A'],
        'Price-to-Sales Ratio': [psRatio],
        'PS Percentile': ['N/A'],
        'EV/EBITDA': [evToEBIDTA],
        'EV/EBITDA Percentile': ['N/A'],
        'EV/GP': [evToGP],
        'EV/GP Percentile': ['N/A'],
        'RV Score': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    rv_dataframe = pd.concat([rv_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(rv_dataframe)

    Ticker  Stock Price Number of Shares to Buy  Price-to-Earning Ratio  \
0        A     110.1500                     N/A               28.835079   
1      AAL      13.2250                     N/A                3.444010   
2      AAP      58.9600                     N/A               10.183074   
3     AAPL     175.1629                     N/A               29.389748   
4     ABBV     153.4700                     N/A               31.578189   
..     ...          ...                     ...                     ...   
498    YUM     125.3400                     N/A               25.321212   
499    ZBH     118.4300                     N/A               49.552301   
500   ZBRA     230.4900                     N/A               19.784549   
501   ZION      34.4850                     N/A                6.082011   
502    ZTS     177.6300                     N/A               37.395789   

    PE Percentile  Price-to-Book Value PB Percentile  Price-to-Sales Ratio  \
0             N/A    

In [20]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,110.1500,N/A,28.835079,N/A,5.801643,N/A,28.835078,N/A,1.914441e+01,N/A,9.098995e+00,N/A,N/A
1,AAL,13.2250,N/A,3.444010,N/A,NaN,N/A,3.444011,N/A,5.240561e+00,N/A,4.333597e+00,N/A,N/A
2,AAP,58.9600,N/A,10.183074,N/A,1.278792,N/A,10.183074,N/A,8.981198e+00,N/A,1.577023e+00,N/A,N/A
3,AAPL,175.1629,N/A,29.389748,N/A,45.473236,N/A,29.389750,N/A,2.247013e+01,N/A,1.630927e+01,N/A,N/A
4,ABBV,153.4700,N/A,31.578189,N/A,21.052126,N/A,31.578190,N/A,1.108060e+01,N/A,7.960365e+00,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,125.3400,N/A,25.321212,N/A,NaN,N/A,25.321212,N/A,1.969126e+01,N/A,1.420128e+01,N/A,N/A
499,ZBH,118.4300,N/A,49.552301,N/A,1.998819,N/A,49.552300,N/A,1.285927e+01,N/A,6.167244e+00,N/A,N/A
500,ZBRA,230.4900,N/A,19.784549,N/A,3.954466,N/A,19.784549,N/A,1.274976e+01,N/A,5.398242e+00,N/A,N/A
501,ZION,34.4850,N/A,6.082011,N/A,1.054877,N/A,6.082011,N/A,8.227780e+09,N/A,8.227780e+09,N/A,N/A


In [21]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
1,AAL,13.225,N/A,3.444010,N/A,NaN,N/A,3.444011e+00,N/A,5.240561,N/A,4.333597e+00,N/A,N/A
52,AZO,2546.340,N/A,18.423703,N/A,NaN,N/A,1.842370e+01,N/A,14.207104,N/A,6.149497e+00,N/A,N/A
53,BA,201.080,N/A,-26.882353,N/A,NaN,N/A,-2.688235e+01,N/A,148.649526,N/A,4.563057e+01,N/A,N/A
57,BBWI,34.420,N/A,11.397351,N/A,NaN,N/A,1.139735e+01,N/A,8.812170,N/A,3.971568e+00,N/A,N/A
66,BKNG,3042.095,N/A,27.072128,N/A,NaN,N/A,2.707213e+01,N/A,17.168797,N/A,1.079231e+11,N/A,N/A
78,CAH,89.630,N/A,89.630000,N/A,NaN,N/A,8.963000e+01,N/A,9.624137,N/A,3.439487e+00,N/A,N/A
80,CAT,274.600,N/A,274.600000,N/A,NaN,N/A,1.417886e+11,N/A,NaN,N/A,NaN,N/A,N/A
98,CL,73.575,N/A,41.103352,N/A,NaN,N/A,4.110335e+01,N/A,17.403611,N/A,6.706368e+00,N/A,N/A
142,DPZ,379.755,N/A,28.682402,N/A,NaN,N/A,2.868240e+01,N/A,21.639694,N/A,1.120762e+01,N/A,N/A
171,ETSY,64.770,N/A,-11.343257,N/A,NaN,N/A,-1.134326e+01,N/A,21.477430,N/A,5.094549e+00,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data which we have to fill with `mean` values.

In [22]:
for column in ['Price-to-Earning Ratio', 'Price-to-Book Value', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

In [23]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

In [24]:
metrics = {
    'Price-to-Earning Ratio': 'PE Percentile',
    'Price-to-Book Value': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

In [25]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,110.1500,N/A,28.835079,0.691849,5.801643,0.671968,28.835078,0.67992,1.914441e+01,0.717694,9.098995e+00,0.465209,N/A
1,AAL,13.2250,N/A,3.444010,0.081511,10.635283,0.850895,3.444011,0.081511,5.240561e+00,0.055666,4.333597e+00,0.149105,N/A
2,AAP,58.9600,N/A,10.183074,0.186879,1.278792,0.105368,10.183074,0.184891,8.981198e+00,0.178926,1.577023e+00,0.017893,N/A
3,AAPL,175.1629,N/A,29.389748,0.703777,45.473236,0.976143,29.389750,0.695825,2.247013e+01,0.809145,1.630927e+01,0.763419,N/A
4,ABBV,153.4700,N/A,31.578189,0.751491,21.052126,0.954274,31.578190,0.739563,1.108060e+01,0.304175,7.960365e+00,0.395626,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,125.3400,N/A,25.321212,0.59841,10.635283,0.850895,25.321212,0.592445,1.969126e+01,0.743539,1.420128e+01,0.709742,N/A
499,ZBH,118.4300,N/A,49.552301,0.900596,1.998819,0.272366,49.552300,0.894632,1.285927e+01,0.413519,6.167244e+00,0.270378,N/A
500,ZBRA,230.4900,N/A,19.784549,0.473161,3.954466,0.544732,19.784549,0.467197,1.274976e+01,0.407555,5.398242e+00,0.212724,N/A
501,ZION,34.4850,N/A,6.082011,0.105368,1.054877,0.059642,6.082011,0.105368,8.227780e+09,0.95825,8.227780e+09,0.910537,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [26]:
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

In [27]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,110.1500,N/A,28.835079,0.691849,5.801643,0.671968,28.835078,0.67992,1.914441e+01,0.717694,9.098995e+00,0.465209,0.645328
1,AAL,13.2250,N/A,3.444010,0.081511,10.635283,0.850895,3.444011,0.081511,5.240561e+00,0.055666,4.333597e+00,0.149105,0.243738
2,AAP,58.9600,N/A,10.183074,0.186879,1.278792,0.105368,10.183074,0.184891,8.981198e+00,0.178926,1.577023e+00,0.017893,0.134791
3,AAPL,175.1629,N/A,29.389748,0.703777,45.473236,0.976143,29.389750,0.695825,2.247013e+01,0.809145,1.630927e+01,0.763419,0.789662
4,ABBV,153.4700,N/A,31.578189,0.751491,21.052126,0.954274,31.578190,0.739563,1.108060e+01,0.304175,7.960365e+00,0.395626,0.629026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,125.3400,N/A,25.321212,0.59841,10.635283,0.850895,25.321212,0.592445,1.969126e+01,0.743539,1.420128e+01,0.709742,0.699006
499,ZBH,118.4300,N/A,49.552301,0.900596,1.998819,0.272366,49.552300,0.894632,1.285927e+01,0.413519,6.167244e+00,0.270378,0.550298
500,ZBRA,230.4900,N/A,19.784549,0.473161,3.954466,0.544732,19.784549,0.467197,1.274976e+01,0.407555,5.398242e+00,0.212724,0.421074
501,ZION,34.4850,N/A,6.082011,0.105368,1.054877,0.059642,6.082011,0.105368,8.227780e+09,0.95825,8.227780e+09,0.910537,0.427833


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [28]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)

In [29]:
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)

In [30]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MHK,90.0700,N/A,-18.232794,0.039761,0.698948,0.019881,-18.232793,0.039761,6.265588e+00,0.089463,2.932975e+00,0.065606,0.050895
1,C,41.9750,N/A,6.652139,0.12326,0.428872,0.005964,6.652139,0.12326,-3.262158e+11,0.001988,-3.262158e+11,0.001988,0.051292
2,EQT,39.2550,N/A,4.470957,0.085487,1.182486,0.087475,4.470957,0.085487,2.958875e+00,0.023857,2.432753e+00,0.043738,0.065209
3,MOS,37.4250,N/A,5.847656,0.10338,1.002330,0.05169,5.847656,0.10338,4.152391e+00,0.039761,2.718194e+00,0.049702,0.069583
4,T,15.3300,N/A,-13.102564,0.04175,1.075563,0.069583,-13.102564,0.04175,6.190190e+00,0.083499,3.844236e+00,0.129225,0.073161
5,PARA,13.4350,N/A,-5.417339,0.067594,0.403890,0.003976,-5.865619,0.065606,9.933107e+00,0.220676,2.357776e+00,0.037773,0.079125
6,STT,68.9800,N/A,9.334235,0.176938,0.999580,0.047714,9.334235,0.172962,-3.643621e+10,0.011928,-3.643621e+10,0.011928,0.084294
7,WRK,35.3800,N/A,-6.374775,0.06163,0.906088,0.035785,-6.618773,0.06163,6.406270e+00,0.101392,4.689944e+00,0.16501,0.085089
8,VTRS,9.7775,N/A,6.349026,0.115308,0.562507,0.007952,6.349026,0.115308,5.823373e+00,0.071571,4.596584e+00,0.161034,0.094235
9,IVZ,14.7800,N/A,10.557143,0.194831,0.585834,0.00994,10.557143,0.192843,5.783173e+00,0.069583,1.640737e+00,0.021869,0.097813


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [31]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [32]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
position_size

200000.0

In [33]:
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[row, 'Stock Price'])

In [34]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MHK,90.0700,15111,-18.232794,0.039761,0.698948,0.019881,-18.232793,0.039761,6.265588e+00,0.089463,2.932975e+00,0.065606,0.050895
1,C,41.9750,4806,6.652139,0.12326,0.428872,0.005964,6.652139,0.12326,-3.262158e+11,0.001988,-3.262158e+11,0.001988,0.051292
2,EQT,39.2550,5088,4.470957,0.085487,1.182486,0.087475,4.470957,0.085487,2.958875e+00,0.023857,2.432753e+00,0.043738,0.065209
3,MOS,37.4250,6048,5.847656,0.10338,1.002330,0.05169,5.847656,0.10338,4.152391e+00,0.039761,2.718194e+00,0.049702,0.069583
4,T,15.3300,1352,-13.102564,0.04175,1.075563,0.069583,-13.102564,0.04175,6.190190e+00,0.083499,3.844236e+00,0.129225,0.073161
5,PARA,13.4350,1174,-5.417339,0.067594,0.403890,0.003976,-5.865619,0.065606,9.933107e+00,0.220676,2.357776e+00,0.037773,0.079125
6,STT,68.9800,1657,9.334235,0.176938,0.999580,0.047714,9.334235,0.172962,-3.643621e+10,0.011928,-3.643621e+10,0.011928,0.084294
7,WRK,35.3800,4520,-6.374775,0.06163,0.906088,0.035785,-6.618773,0.06163,6.406270e+00,0.101392,4.689944e+00,0.16501,0.085089
8,VTRS,9.7775,1285,6.349026,0.115308,0.562507,0.007952,6.349026,0.115308,5.823373e+00,0.071571,4.596584e+00,0.161034,0.094235
9,IVZ,14.7800,1995,10.557143,0.194831,0.585834,0.00994,10.557143,0.192843,5.783173e+00,0.069583,1.640737e+00,0.021869,0.097813


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [35]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Formatting Our Excel Output

In [36]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_template = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

float_template = writer.book.add_format(
    {
        'num_format': '0.0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_template = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [37]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Stock Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earning Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Value', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

In [38]:
writer.close()